In [ ]:
import findspark
findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand
from pyspark.sql.types import StructType, StructField, FloatType
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.linalg import DenseVector
from pyspark.streaming import StreamingContext
import time



spark = SparkSession.builder.appName("analytics").getOrCreate()
sc = spark.sparkContext

# Configuramos el contexto de streaming con un intervalo de 1 segundo
ssc = StreamingContext(sc, 1)

In [5]:
!which python3

/opt/homebrew/bin/python3


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Iniciamos la sesión de Spark
spark = SparkSession.builder \
    .appName("analytics") \
    .getOrCreate()

# Definimos el esquema del dataset simulado
schema = StructType([
    StructField("age", FloatType(), True),
    StructField("creatinine_phosphokinase", FloatType(), True),
    StructField("ejection_fraction", FloatType(), True),
    StructField("platelets", FloatType(), True),
    StructField("serum_creatinine", FloatType(), True),
    StructField("serum_sodium", FloatType(), True),
    StructField("time", FloatType(), True)
])

# Simulamos un dataset para entrenar el modelo de escalado
def generate_training_data():
    return spark.range(0, 1000).select(
        (rand() * 100).alias("age"),
        (rand() * 8000).alias("creatinine_phosphokinase"),
        (rand() * 80).alias("ejection_fraction"),
        (rand() * 450000).alias("platelets"),
        (rand() * 5).alias("serum_creatinine"),
        (rand() * 150).alias("serum_sodium"),
        (rand() * 300).alias("time")
    )

training_data = generate_training_data()

# Seleccionamos las características que vamos a utilizar para el escalado
features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']

# Vectorizamos las características seleccionadas
vector_assembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
training_data_vector = vector_assembler.transform(training_data)

# Entrenamos el StandardScaler
scaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
scaler_model = scaler.fit(training_data_vector)

# Guardamos el modelo de escalado
scaler_model.write().overwrite().save("data/scaler_model")

# Streaming K-Means

### ¿Qué es y para qué se usa Streaming K-Means?
**Streaming K-Means** es una variante del algoritmo de clustering K-Means diseñado para trabajar con datos que llegan en un flujo continuo (streaming). A diferencia del K-Means tradicional, que opera en un conjunto de datos estático, Streaming K-Means permite actualizar los centroides de los clusters en tiempo real conforme nuevos datos llegan, sin necesidad de almacenar todos los datos en memoria. Esto es particularmente útil en aplicaciones donde los datos son generados de forma continua, como monitoreo de sensores, análisis de tráfico de redes, procesamiento de logs en tiempo real, entre otros.

### ¿Cómo funciona?
El funcionamiento de Streaming K-Means se basa en actualizar iterativamente los centroides de los clusters a medida que nuevos datos son procesados. Uno de los parámetros clave en este proceso es el decayFactor, que controla la influencia de los datos nuevos en la actualización de los centroides. El decayFactor determina la tasa de olvido, es decir, cuánto peso se da a los datos recientes en comparación con los datos históricos. Un decayFactor cercano a 1 implica que los datos nuevos tienen un peso significativo y los centroides se adaptan rápidamente a los cambios recientes, mientras que un valor cercano a 0 implica que los datos históricos tienen más peso y los centroides cambian más lentamente. Este balance permite ajustar el modelo según las necesidades específicas de la aplicación, ya sea para captar rápidamente cambios en los patrones o para mantener estabilidad en clusters bien definidos.

### ¿En qué escenarios se puede aplicar?

* Deteccion de fraudes monitoreando transacciones
* Deteccion de anomalias en transito de red
* Monitoreo de sensores de IoT

## Generación de Datos sintéticos:

Para poder simular el flujo de información

In [ ]:
# Definimos el esquema del dataset simulado
schema = StructType([
    StructField("age", FloatType(), True),
    StructField("creatinine_phosphokinase", FloatType(), True),
    StructField("ejection_fraction", FloatType(), True),
    StructField("platelets", FloatType(), True),
    StructField("serum_creatinine", FloatType(), True),
    StructField("serum_sodium", FloatType(), True),
    StructField("time", FloatType(), True)
])

# Simulamos un stream de datos creando un DataFrame con valores aleatorios
def generate_stream_data():
    return spark.range(0, 100).select(
        (rand() * 100).alias("age"),
        (rand() * 8000).alias("creatinine_phosphokinase"),
        (rand() * 80).alias("ejection_fraction"),
        (rand() * 450000).alias("platelets"),
        (rand() * 5).alias("serum_creatinine"),
        (rand() * 150).alias("serum_sodium"),
        (rand() * 300).alias("time")
    )

# Entrenamos el StandardScaler y guardamos el modelo
def train_and_save_scaler_model():
    training_data = generate_stream_data()
    vector_assembler = VectorAssembler(inputCols=['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time'], outputCol="unscaled_features")
    training_data_vector = vector_assembler.transform(training_data)
    scaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
    scaler_model = scaler.fit(training_data_vector)
    scaler_model.write().overwrite().save("data/scaler_model")
    return scaler_model

scaler_model = train_and_save_scaler_model()

# Inicializamos el VectorAssembler y cargamos el modelo de escalado
vector_assembler = VectorAssembler(inputCols=['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time'], outputCol="unscaled_features")
scaler_model = StandardScaler.load('data/scaler_model')

# Función para simular la generación de datos en un DStream
def stream_data_generator():
    while True:
        new_data = generate_stream_data()
        yield new_data

        
        
# Generamos un DStream simulado
input_data = ssc.queueStream([sc.parallelize(generate_stream_data().select("age", "creatinine_phosphokinase", "ejection_fraction", "platelets", "serum_creatinine", "serum_sodium", "time").collect()) for _ in range(5)])


## Procesamiento de datos

In [ ]:
# Transformamos los datos usando VectorAssembler y StandardScalerModel directamente
def transform(rdd):
    if not rdd.isEmpty():
        df = rdd.toDF(schema)
        df_vector = vector_assembler.transform(df)
        df_scaled = scaler_model.transform(df_vector)
        return df_scaled.select("features").rdd.map(lambda row: DenseVector(row["features"]))
    else:
        return rdd

data_stream = input_data.transform(transform)

## Entrenamiento del modelo


## StreamingKMeans

Definimos el modelo StreamingKMeans

In [ ]:
# Definimos el modelo StreamingKMeans
initCenters = [[0.0] * 7, [1.0] * 7, [2.0] * 7]
initWeights = [1.0, 1.0, 1.0]
streaming_kmeans = StreamingKMeans(k=3, decayFactor=0.5, timeUnit="points").setInitialCenters(initCenters, initWeights)

# Entrenamos el modelo inicial con el DStream
streaming_kmeans.trainOn(data_stream)

# Función para mostrar los centroides actuales
def print_centroids(model):
    centers = model.latestModel().centers
    print("Cluster Centers: ")
    for center in centers:
        print(center)

# Mostramos los centroides iniciales
print("Initial cluster centers:")
print_centroids(streaming_kmeans)


## Actualización del modelo

Con el primer "batch" de datos que llega por la red

In [ ]:
# Iniciamos el contexto de streaming
ssc.start()
time.sleep(10)  # Esperamos un tiempo para permitir que los datos se procesen
ssc.stop(stopSparkContext=True, stopGraceFully=True)

# Mostramos los centroides finales
print("Final cluster centers:")
print_centroids(streaming_kmeans)

In [ ]:
# Mostramos los centroides finales
print("Final cluster centers:")
print_centroids(streaming_kmeans)

## Simulación del proceso de actualización

In [ ]:
# Mostramos los centroides iniciales
print("Initial cluster centers:")
print_centroids(model)

# Función para actualizar el modelo con nuevos datos en el stream y mostrar los centroides
def update_model_and_print_centroids():
    new_data = generate_stream_data()
    new_data_scaled = pipeline_model.transform(new_data)
    model.trainOn(new_data_scaled)
    print("Updated cluster centers:")
    print_centroids(model)

# Simulamos la actualización del modelo con nuevos datos
for _ in range(5):  # Simulamos 5 batches de datos
    update_model_and_print_centroids()

## Evaluación del modelo

* Realiza predicciones en el conjunto de prueba.
* Evalúa el modelo utilizando la métrica de RMSE.

In [ ]:
# Hacer predicciones
predictions = model.transform(test_data)

# Evaluar el modelo
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) en el conjunto de prueba: {rmse}")


## Guardar y cargar el modelo

In [ ]:
# Guardar el modelo
pipelinePath = "data/lr-pipeline-model"
pipelineModel.write().overwrite().save(pipelinePath)


# Cargar el modelo
from pyspark.ml import PipelineModel 
savedPipelineModel = PipelineModel.load(pipelinePath)